# Step 1: Extract Road Network from OpenStreetMap (OSM)
This notebook will show you how to go from an .osm.pbf file to a GOSTnets-compatible network format. 
In addition, this notebook will show you how to:
- Clip the roads to an input polygon (here we demonstrate with an ESRI Shapefile, rek2.shp)
- Filter the roads DataFrame by a list of accepted road types, as tagged in OSM (see the list `accepted_road_types` that we define in the code)

## Import Libraries
First, we import the necessary libraries.
In this notebook, we use the following libraries:
- `geopandas` to manipulate geospatial data (e.g. clipping the roads to a polygon)
- `os` to define the path to the .osm.pbf file
- `GOSTnets` to import the OSM data and to save the network in a GOSTnets-compatible format (.gpickle in this case)

In [ ]:
import geopandas as gpd
import os
import GOSTnets as gn

The load_osm module is an optional GOSTNets submodule that is used to import OSM data. Make sure gdal, geopy, and boltons are installed first as it has additional dependencies (look in GOSTnets repo main README file if you need additional help with this). Then import the submodule.

In [ ]:
from GOSTnets.load_osm import OSM_to_network

## Download data and set filepaths
We will use the `osm.pbf` file for the entire country of Iceland. You can download the file from [Geofabrik](https://download.geofabrik.de/europe/iceland.html), if you didn't download it from the GOSTnets repository "tutorial_data" folder.
Once you have downloaded the file, set the filepath to the .osm.pbf file, this can be a relative or absolute path.
We demonstrate construction of a relative file path using the `os.path` module to define a relative path to the "tutorial_data" folder.

In [ ]:
pth = "./"  # change this path to your working folder
fil = r"iceland-latest.osm.pbf"  # download this file from geofabrik: http://download.geofabrik.de/europe/iceland.html.

# be sure to place the .osm.pbf file in the 'tutorial data' folder.

f = os.path.join(pth, "tutorial_data", fil)

In [ ]:
# check if the file exists, raise an exception if it does not
if os.path.isfile(f):
    print("File found.")
else:
    raise Exception("File not found.")
# print the file path
print(f)

## Load the OSM data as a GOSTnets object

GOSTNets can load OSM data as a special 'OSM_to_network' object.
This object gets initialized with both a copy of the OSM file itself and the roads extracted from the OSM file in a GeoPandas DataFrame. 
This DataFrame is a property of the object called 'roads_raw' and is the starting point for our network.

In [ ]:
# convert the .osm.pbf file to a GOSTnets object
iceland = OSM_to_network(f)

In [ ]:
# show the doc-string for the OSM_to_network object
?iceland

In [ ]:
# display the first 5 rows of the extracted dataframe of the roads network
iceland.roads_raw.head()

In [ ]:
# show the different road types and counts
iceland.roads_raw.infra_type.value_counts()

### Define the accepted road types list

We define a list of the types of roads from the above that we consider acceptable for our road network.
This list is used to filter the roads DataFrame to only include the types specified.

In [ ]:
accepted_road_types = [
    "residential",
    "unclassified",
    "track",
    "service",
    "tertiary",
    "road",
    "secondary",
    "primary",
    "trunk",
    "primary_link",
    "trunk_link",
    "tertiary_link",
    "secondary_link",
]

### Filter the roads DataFrame

We call the `filterRoads` method and pass it a list of acceptable road types.
Then we can look at the infrastructure types in the filtered roads DataFrame (similar to what was done above), except now we have filtered the roads to only include the types we want.

In [ ]:
iceland.filterRoads(acceptedRoads=accepted_road_types)

In [ ]:
iceland.roads_raw.infra_type.value_counts()

## Clip the roads to a polygon

Here, we import a shapefile for our area of interest. We want to make a map of Reykjavik, Iceland. We clip the extent of the country file to just the area around the capital (rek2.shp) below using standard GeoPandas functions:

In [ ]:
# read the shapefile for the clip area
clip_shp = gpd.read_file(os.path.join(pth, "tutorial_data", "rek2.shp"))
clip_shp = clip_shp.to_crs("epsg:4326")
clip_shp_obj = clip_shp.geometry.iloc[0]

In [ ]:
# visualize the clip area
clip_shp.plot()

In [ ]:
# this is the geometry object (shapely) from clip_shp
clip_shp_obj

### Check to ensure some roads intersect the AOI

We check to see everything lines up by running intersect and counting the True / False returns.
The count of the True values are the number of roads that intersect the AOI

In [ ]:
# intersects is a Shapely function that returns True if the boundary or interior of the object intersect in any way with those of the other
iceland.roads_raw.geometry.intersects(clip_shp_obj).value_counts()

Next we remove any roads that don't intersect the capital area, by modifying the `roads_raw` DataFrame contained within the `OSM_to_network` object 

In [ ]:
iceland.roads_raw = iceland.roads_raw.loc[
    iceland.roads_raw.geometry.intersects(clip_shp_obj) == True  # noqa: E712
]

In [ ]:
# reprint the intersects value counts (should only be True now)
iceland.roads_raw.geometry.intersects(clip_shp_obj).value_counts()

### Generate the RoadsGPD object

Having narrowed the DataFrame to the roads we are interested in, we generate the `RoadsGPD` object, which is stored as a property of the `OSM_to_network` object.
The `RoadsGPD` object is a GeoDataFrame that further processes the roads.
This includes splitting the edges where intersections occur, adding unique edge IDs, and adding to/from columns to the GeoDataFrame.

In [ ]:
# generate the roads GeoDataFrame, may take a few minutes
iceland.generateRoadsGDF(verbose=False)

In [ ]:
# view the first 5 rows of the newly created roads GeoDataFrame
iceland.roadsGPD.head()

### Generate the GOSTnets graph object

With the `initialReadIn()` method, we transform the roads GeoDataFrame into a graph object

In [ ]:
iceland.initialReadIn()

## Save the graph object to file

We save this graph object down to file using `gn.save()`. The save function produces three outputs: a node GeoDataFrame as a CSV, an edge GeoDataFrame as a CSV, and a graph object saved as a pickle.

In [ ]:
gn.save?

In [ ]:
gn.save(iceland.network, "iceland_unclean", os.path.join(pth, "tutorial_outputs"))

Move on to [Step 2](Step_2-Clean_Network.ipynb) to see how we clean up this rough network.